In [4]:
import pyforest
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width',1000)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
loan=pd.read_csv('loan.csv')

<IPython.core.display.Javascript object>

In [16]:
loan1=loan.copy()
# impute gender with most freq category
loan1['Gender']=loan1['Gender'].fillna('Male')
# impute gender with most freq category
loan1['Dependents']=loan1['Dependents'].fillna('0')
# impute gender with most freq category
loan1['Self_Employed']=loan1['Self_Employed'].fillna('No')
# as the distribution is skewed, we'll do for median imputation
loan1['LoanAmount']=loan1['LoanAmount'].fillna(loan1['LoanAmount'].median())
# we'll do for mean imputation
loan1['Loan_Amount_Term']=loan1['Loan_Amount_Term'].fillna(loan1['Loan_Amount_Term'].mean())
# impute gender with most freq category
loan1['Credit_History']=loan1['Credit_History'].fillna(1.0)
# impute gender with most freq category
loan1['Married']=loan1['Married'].fillna('Yes')
# drop loanid
loan1.drop('Loan_ID',axis=1,inplace=True)

In [17]:
from sklearn.preprocessing import LabelEncoder
def le(df,col):
    lab=LabelEncoder()
    df[col]=pd.Series(lab.fit_transform(df[col]))

In [18]:
le(loan1,'Gender')
le(loan1,'Married')
le(loan1,'Dependents')
le(loan1,'Education')
le(loan1,'Self_Employed')
le(loan1,'Property_Area')
le(loan1,'Loan_Status')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
from scipy.stats import zscore
z=np.abs(zscore(loan1))
loan1=loan1[(z<3).all(axis=1)]

x=loan1.drop('Loan_Status',axis=1)
y=loan1['Loan_Status']

<IPython.core.display.Javascript object>

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score,cross_val_predict,cross_validate

# Now, let's define some functions to reuse them
def fitc(m,x,y,rs=45):
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=rs,stratify=y)
    m.fit(x_train,y_train)
    
def get_scorec(m,x,y,rs=45):
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=rs,stratify=y)
    m.fit(x_train,y_train)
    train_score=m.score(x_train,y_train)
    pred=m.predict(x_test)
    test_score=accuracy_score(pred,y_test)
    return train_score,test_score

def get_predc(m,x,y,rs=45):
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=rs,stratify=y)
    m.fit(x_train,y_train)
    pred=m.predict(x_test)
    return pred

def get_metricsc(m,x,y,rs=45):
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=rs,stratify=y)
    m.fit(x_train,y_train)
    pred=m.predict(x_test)
    acc=accuracy_score(pred,y_test)
    conf=confusion_matrix(pred,y_test)
    clas=classification_report(pred,y_test)
    return acc,conf,clas

from sklearn.model_selection import GridSearchCV
def gridcvc(m,p,x,y):
    gkb=GridSearchCV(m,p,n_jobs=-1,cv=5,scoring='accuracy')
    gkb.fit(x,y)
    return gkb.best_params_


from sklearn.model_selection import RandomizedSearchCV
def randomcvc(m,p,x,y):
    from sklearn.model_selection import RandomizedSearchCV
    gkb=RandomizedSearchCV(estimator=m,param_distributions=p,
                          cv=3,n_iter=50,
                          n_jobs=-1, verbose=5,return_train_score=True,random_state=45)
    gkb.fit(x,y)
    return gkb.best_params_

from mlxtend.feature_selection import SequentialFeatureSelector as sfs
def bfec(m,x,y,k):
    sf=sfs(m, k_features=k, forward=False, cv=5, n_jobs=-1)
    sf.fit(x,y)
    feat_names=list(sf.k_feature_names_)
    xbfe=x[feat_names]
    return xbfe

def ffsc(m,x,y,k):
    sf=sfs(m, k_features=k, forward=True, cv=5, n_jobs=-1)
    sf.fit(x,y)
    feat_names=list(sf.k_feature_names_)
    xffs=x[feat_names]
    return xffs

from sklearn.decomposition import PCA
def nforpcac(x):
    pca=PCA().fit(x)
    plt.figure()
    plt.plot(np.cumsum(pca.explained_variance_ratio_)*100)
    plt.xlabel('No of Components')
    plt.ylabel('Variance')
    plt.axhline(y=95, color='r', linestyle='--')
    plt.show()
    
def pcac(m,x,n):
    pca=PCA(n_components=n)
    xpca=pd.DataFrame(pca.fit_transform(x))
    return xpca

def crossc(m,x,y,c=5):
    score=cross_val_score(m,x,y,cv=c,n_jobs=-1)
    return (score.mean(), score.std())

from sklearn.preprocessing import StandardScaler
def stdc(x):
    scx=StandardScaler()
    xs=pd.DataFrame(scx.fit_transform(x), columns=x.columns)
    return xs

from sklearn.preprocessing import MinMaxScaler
def mmc(x):
    mm=MinMaxScaler()
    xm=pd.DataFrame(mm.fit_transform(x), columns=x.columns)
    return xm

def modified_bfec(m,x,y,k):
    sf=sfs(m, k_features=k, forward=False, cv=5, n_jobs=-1)
    sf.fit(x,y)
    feat_names=list(sf.k_feature_names_)
    xbfe=x[feat_names]
    x_train, x_test, y_train, y_test=train_test_split(xbfe,y,test_size=0.25,random_state=45,stratify=y)
    m.fit(x_train,y_train)
    pred=m.predict(x_test)
    test_score=accuracy_score(pred,y_test)
    return test_score

def modified_ffsc(m,x,y,k):
    sf=sfs(m, k_features=k, forward=True, cv=5, n_jobs=-1)
    sf.fit(x,y)
    feat_names=list(sf.k_feature_names_)
    xffs=x[feat_names]
    x_train, x_test, y_train, y_test=train_test_split(xffs,y,test_size=0.25,random_state=45,stratify=y)
    m.fit(x_train,y_train)
    pred=m.predict(x_test)
    test_score=accuracy_score(pred,y_test)
    return test_score

def modified_pcac(m,x,y,n):
    pca=PCA(n_components=n)
    xpca=pd.DataFrame(pca.fit_transform(x))
    x_train, x_test, y_train, y_test=train_test_split(xpca,y,test_size=0.25,random_state=45,stratify=y)
    m.fit(x_train,y_train)
    pred=m.predict(x_test)
    test_score=accuracy_score(pred,y_test)
    return test_score

def ttsc(x,y,rs=45):
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=rs,stratify=y)
    return x_train, x_test, y_train, y_test

def classification(m,x,y,rs=45):
    x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=rs,stratify=y)
    m.fit(x_train,y_train)
    pred=m.predict(x_test)
    clas=classification_report(pred,y_test)
    return clas

lrc=LogisticRegression(n_jobs=-1,random_state=45)
dtc=DecisionTreeClassifier(random_state=45)
svc=SVC(random_state=45)
knnc=KNeighborsClassifier(n_jobs=-1)
rfc=RandomForestClassifier(n_jobs=-1,random_state=45)
gbc=GradientBoostingClassifier(random_state=45)
xgbc=XGBClassifier(n_jobs=-1,random_state=45)
adac=AdaBoostClassifier(random_state=45)
gnb=GaussianNB()
mnb=MultinomialNB()

# Iteration 1 

In [21]:
# lets check the scores of all models on the data

In [22]:
print("LR ", get_scorec(lrc,x,y))
print("DT ", get_scorec(dtc,x,y))
print("SVC ", get_scorec(svc,x,y))
print("KNN ", get_scorec(knnc,x,y))
print("GNB ", get_scorec(gnb,x,y))
print("MNB ", get_scorec(mnb,x,y))
print("RF ", get_scorec(rfc,x,y))
print("GB ", get_scorec(gbc,x,y))
print("XGB ", get_scorec(xgbc,x,y))

LR  (0.8194444444444444, 0.8137931034482758)
DT  (1.0, 0.6896551724137931)
SVC  (0.9976851851851852, 0.6896551724137931)
KNN  (0.7222222222222222, 0.6551724137931034)
GNB  (0.8101851851851852, 0.7931034482758621)
MNB  (0.5092592592592593, 0.5724137931034483)
RF  (0.9861111111111112, 0.7448275862068966)
GB  (0.9050925925925926, 0.7931034482758621)
XGB  (1.0, 0.7862068965517242)


In [23]:
# models are over fitting

# Iteration 2

In [24]:
xs=stdc(x)

<IPython.core.display.Javascript object>

In [26]:
print("LR ", get_scorec(lrc,xs,y))
print("DT ", get_scorec(dtc,xs,y))
print("SVC ", get_scorec(svc,xs,y))
print("KNN ", get_scorec(knnc,xs,y))
print("GNB ", get_scorec(gnb,xs,y))
# print("MNB ", get_scorec(mnb,xs,y))
print("RF ", get_scorec(rfc,xs,y))
print("GB ", get_scorec(gbc,xs,y))
print("XGB ", get_scorec(xgbc,xs,y))

LR  (0.8194444444444444, 0.8137931034482758)
DT  (1.0, 0.6896551724137931)
SVC  (0.8356481481481481, 0.8137931034482758)
KNN  (0.8379629629629629, 0.7931034482758621)
GNB  (0.8171296296296297, 0.8)
RF  (0.9861111111111112, 0.7448275862068966)
GB  (0.9050925925925926, 0.7931034482758621)
XGB  (1.0, 0.7862068965517242)


# Iteration 3

In [27]:
# lets remove skewness

In [28]:
x.skew()

Gender              -1.622920
Married             -0.630211
Dependents           1.052106
Education            1.306588
Self_Employed        2.252848
ApplicantIncome      2.148522
CoapplicantIncome    1.350517
LoanAmount           1.151525
Loan_Amount_Term    -2.077031
Credit_History      -1.976043
Property_Area       -0.055332
dtype: float64

In [29]:
x_sk=x.copy()

In [34]:
# x_sk['ApplicantIncome'].min(), x_sk['ApplicantIncome'].max()
x_sk['ApplicantIncome']=np.log(x_sk['ApplicantIncome'])
# skewness reduced to -0.20

<IPython.core.display.Javascript object>

In [37]:
# x_sk['CoapplicantIncome'].min(), x_sk['CoapplicantIncome'].max()
x_sk['CoapplicantIncome']=np.cbrt(x_sk['CoapplicantIncome'])
# skewness reduced to -0.03

<IPython.core.display.Javascript object>

In [41]:
# x_sk['LoanAmount'].min(), x_sk['LoanAmount'].max()
x_sk['LoanAmount']=np.cbrt(x_sk['LoanAmount'])
# skewness reduced to 0.05

<IPython.core.display.Javascript object>

In [52]:
# x_sk['Loan_Amount_Term'].min(), x_sk['Loan_Amount_Term'].max()
x_sk['Loan_Amount_Term']=np.power(x_sk['Loan_Amount_Term'],2.5)
# skewness reduced to 0.05

<IPython.core.display.Javascript object>

In [57]:
x_sk['Credit_History'].min(), x_sk['Credit_History'].max()
np.power(x_sk['Credit_History'],3).skew()
# skewness remained

<IPython.core.display.Javascript object>

-1.976042837301836

In [58]:
x_sk['Self_Employed'].min(), x_sk['Self_Employed'].max()
# x_sk['Self_Employed']=np.power(x_sk['Self_Employed'],2.5)
# skewness reduced to 0.05

(0, 1)

In [59]:
x_sk['Education'].min(), x_sk['Education'].max()
# x_sk['Education']=np.power(x_sk['Education'],2.5)
# skewness reduced to 0.05

(0, 1)

In [67]:
# x_sk['Dependents'].min(), x_sk['Dependents'].max()
x_sk['Dependents']=np.power(x_sk['Dependents'],1/5)
# skewness reduced to 0.43

<IPython.core.display.Javascript object>

In [68]:
x_sk.skew()

Gender              -1.622920
Married             -0.630211
Dependents           0.437276
Education            1.306588
Self_Employed        2.252848
ApplicantIncome     -0.207967
CoapplicantIncome   -0.038951
LoanAmount           0.054080
Loan_Amount_Term    -0.125508
Credit_History      -1.976043
Property_Area       -0.055332
dtype: float64

In [69]:
print("LR ", get_scorec(lrc,x_sk,y))
print("DT ", get_scorec(dtc,x_sk,y))
print("SVC ", get_scorec(svc,x_sk,y))
print("KNN ", get_scorec(knnc,x_sk,y))
print("GNB ", get_scorec(gnb,x_sk,y))
print("MNB ", get_scorec(mnb,x_sk,y))
print("RF ", get_scorec(rfc,x_sk,y))
print("GB ", get_scorec(gbc,x_sk,y))
print("XGB ", get_scorec(xgbc,x_sk,y))

LR  (0.6898148148148148, 0.6896551724137931)
DT  (1.0, 0.696551724137931)
SVC  (0.8240740740740741, 0.7724137931034483)
KNN  (0.7870370370370371, 0.7103448275862069)
GNB  (0.6759259259259259, 0.6551724137931034)
MNB  (0.7407407407407407, 0.7310344827586207)
RF  (0.9861111111111112, 0.7379310344827587)
GB  (0.9050925925925926, 0.7931034482758621)
XGB  (1.0, 0.7862068965517242)


# Iteration 4

In [70]:
xs_sk=stdc(x_sk)

<IPython.core.display.Javascript object>

In [71]:
print("LR ", get_scorec(lrc,xs_sk,y))
print("DT ", get_scorec(dtc,xs_sk,y))
print("SVC ", get_scorec(svc,xs_sk,y))
print("KNN ", get_scorec(knnc,xs_sk,y))
print("GNB ", get_scorec(gnb,xs_sk,y))
# print("MNB ", get_scorec(mnb,xs_sk,y))
print("RF ", get_scorec(rfc,xs_sk,y))
print("GB ", get_scorec(gbc,xs_sk,y))
print("XGB ", get_scorec(xgbc,xs_sk,y))

LR  (0.8194444444444444, 0.8137931034482758)
DT  (1.0, 0.696551724137931)
SVC  (0.8287037037037037, 0.8206896551724138)
KNN  (0.8240740740740741, 0.7793103448275862)
GNB  (0.8171296296296297, 0.8)
RF  (0.9861111111111112, 0.7379310344827587)
GB  (0.9050925925925926, 0.7931034482758621)
XGB  (1.0, 0.7862068965517242)


# Conclusion

In [72]:
# LR with xs_sk 0.8194444444444444, 0.8137931034482758
# SVC with xs_sk 0.8287037037037037, 0.8206896551724138
# GNB with xs_sk 0.8171296296296297, 0.8
# GB with x 0.9050925925925926, 0.7931034482758621